## Exercise 4

For this exercise you are given a database containing information about real estate around
Uppsala. You are also given a Python module that helps working the database.

- The  database (`uppsala.sqlite`) and the code (`db.py` and `homes.py`) are found in the `download` directory.
- Read the module's [documentation](../../files/db_module_documentation.html)

The module `db.py` will help you produce a HTML file whit your results plotted on a map. This set-up uses OpenStreetMap. 

![result](../../img/mapOSM.png)

The center is defined as the point:

`latitude: 59.865795990339876`

`longitude: 17.64583576202392`


- Open `homes.py` and use it to solve the exercise

### Questions:


a) What is the price of the cheapest house around the given center and within 2 kilometers?

b) Plot the 100 cheapest houses in this area.

c) Find the most expensive house that has a square meter larger than 10, and print its price and square meter.

d) Print or plot this house.

----
<br><br><br><br><br>
<h2>Proposed solution</h2>

**a) and b)**

First, we do all imports and set the coordinates of the 'center'

In [1]:
import db
from imp import reload
reload(db)

<module 'db' from '/home/malin/NBIS/pycourse/PythonCourse/exercises/day4/db.py'>

In [2]:
from db import HomeDB
from db import haversine as get_distance
from db import plot, sort_by_price

lat = 59.865795990339876
lng = 17.64583576202392
radius = 2000  # in m


db = HomeDB('../../files/uppsala.sqlite')

We define a help function that gets all houses that are central and returns them sorted by price:

In [3]:
def cheap_central():
    """ Find the within 2000m from the center and sort by increasing price """
    db.connect()
    homes = db.select()
    db.disconnect()
    # TODO sort homes by price and keep only homes that are close to the center

Let's start by picking homes that are central. We create a new list, and put all central houses in it. But how do we know if a home is central?

- we get it's location
- we use the `get_distance()` from `db` to check the distance between the home and the given center

```py
selected = []
for home in homes:
    h_lat, h_lon = home.get_location()
    distance = get_distance(h_lat, h_lon, lat, lng)
    if distance <= 2000:
        selected.append(home)
```

Next step is to sort the homes by price. The function `sort_by_price()` from `db` is useful here!

```py
sorted_selected = sort_by_price(selected)
```

Let's put it all together:

In [4]:
def cheap_central():
    """ Find the within 2000m from the center and sort by increasing price """
    db.connect()
    homes = db.select()
    db.disconnect()
    selected = []
    for home in homes:
        h_lat, h_lon = home.get_location()
        distance = get_distance(h_lat, h_lon, lat, lng)
        if distance <= 2000:
            selected.append(home)
    sorted_selected = sort_by_price(selected)
    return sorted_selected

For a), we just print the first item of the list

In [5]:
def part_a():
    print('Cheapest house: ' + str(cheap_central()[0].get_price()))

For b), we select the 100 first items and plot them. `outfile` is the name of the resulting html file.

In [6]:
def part_b(outfile):
    num = 100
    central = cheap_central()
    pos = set()
    for home in central[:num]:
        pos.add(home.get_location())
    plot(central[:num],
         output=outfile,
         special=central[0],
         zoom=14,
         latitude=lat,
         longitude=lng,
         radius=radius
    )

**c) and d)**

We define another help function to find the most expensive houses.

In [7]:
def expensive():
    """ Sort houses by price per square meter """
    db.connect()
    homes = db.select()
    db.disconnect()
    selected = []
    for home in homes:  # this time we only want homes with areas larger than 10.
        if home.get_area() > 10:
            selected.append(home)
    expensive = sort_by_price(selected, reverse=True)
    return expensive

For c), we get the first item and print it.

In [8]:
def part_c():
    house = expensive()[0]
    print('Most expensive:' + str(house.get_price()) + '. Square meters: ' +str(house.get_area()))

For d), we plot it. Again, `outfile` is the name of the resulting html file.

In [9]:
def part_d(outfile):
    house = expensive()[0]
    plot([house],
         output=outfile,
         special=house,
         zoom=14,  # We zoom out a bit since the most expensive house may not be central
         latitude=lat,
         longitude=lng,
         radius=radius
    )

Optionally, add a main function to print all answers.

In [10]:
if __name__ == "__main__":
    print('Part a:')
    part_a()
    print()
    part_b('selection_b.html')
    print('Part b: printed to selection_b.html')
    print()
    print('Part c:')
    part_c()
    print()
    part_d('selection_d.html')
    print('Part d: printed to selection_d.html')

Part a:
Cheapest house: 330000.0

Part b: printed to selection_b.html

Part c:
Most expensive:20000000.0. Square meters: 120.0

Part d: printed to selection_d.html


### `selection_b.html`:

![result](../../img/mapOSM.png)

### `selection_d.html`:

![result](../../img/ex4_4_2.png)